# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
# Math and Data Processing
import os
import numpy as np
from matplotlib import pyplot as plt
import time

# Sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report

# Tensor & Keras
from tensorflow.keras.layers import Dense, GRU, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Evaluation
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Record and Testing Realtime
from scipy import stats
import mediapipe as mp
import cv2

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

# 3. Extract Keypoint Values

In [ ]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# 4. Setup Folders for Collection

In [ ]:
# Path for exported data, numpy arrays
# Utility untuk Pembuatan Folder automatis untuk penyimpaman data koordinat lokasi pose, wajah, dan gerakan tangan -> .npy format
# SIBI
DATA_PATH = os.path.join('BIGRU_SIBI_DATASET_9Aug2024') 

# Actions that we try to detect
actions = np.array(['Tolong', 'Terima Kasih', 'Maaf'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length (30x30 = 900 frames)
sequence_length = 30

# augmented data total (30 frames x 12 augmentation + 30 frames original data)
total_augmented_sequence = 390

# Folder start
start_folder = 30

In [ ]:
# RUN 1x UNTUK BUAT FOLDER SAJA (JANGAN DI RUN LAGI) UNTUK RECORDING DATA 
for action in actions: 
    for sequence in range(total_augmented_sequence):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
# RUN 1x UNTUK BUAT FOLDER SAJA (JANGAN DI RUN LAGI) UNTUK RECORDING DATA 
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(0, no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                
                # Read feed
                ret, frame = cap.read()

                # Save Extracted Landmarks to .npy file
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))

                # Save image
                cv2.imwrite(npy_path + '.jpg', frame)

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255, 0), 2, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,75), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Hazlan SIBI Record Webcam', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,75), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Hazlan SIBI Record Webcam', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                # npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

# 6. Data Augmentation

#### Augmented Function Paramater = 12
#### Deleted (rotate: 45, 315 | zoomin: 0.9 | zoomout: 0.9)

In [ ]:
def augmentation_rotation(image, angle):
    # Rotate the image
    array = [15,30,345,330]
    image_list = []
    for element in array:
        rows, cols, _ = image.shape
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), element, 1)
        augmented_image = cv2.warpAffine(image, M, (cols, rows))
        image_list.append(augmented_image)
    # Show to screen
    return image_list

In [ ]:
def zoomin(image, zoom_factor):
    height, width = image.shape[:2]
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    left = int((width - new_width))
    top = int((height - new_height))
    right = int((width + new_width))
    bottom = int((height + new_height))
    cropped_image = image[top:bottom, left:right]
    zoom_in_width = int(image.shape[1])
    zoom_in_height = int(image.shape[0])
    zoom_in_image = cv2.resize(cropped_image, (zoom_in_width, zoom_in_height))
    return zoom_in_image

def augmentation_zoomin(image):
    array = [0.8,0.7]
    image_list = []
    for element in array:
        augmented_image = zoomin(image, element)
        image_list.append(augmented_image)
    return image_list



In [ ]:
def zoomout(image, zoom_factor):
    height, width = image.shape[:2]
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)

    # Compute the aspect ratio difference
    width_ratio = new_width / width
    height_ratio = new_height / height
    aspect_ratio = min(width_ratio, height_ratio)

    # Compute the new dimensions
    new_width = int(width * aspect_ratio)
    new_height = int(height * aspect_ratio)

    # Ensure that the new dimensions are not larger than the original image dimensions
    new_width = min(new_width, width)
    new_height = min(new_height, height)

    # Compute the black border dimensions
    border_width = width - new_width
    border_height = height - new_height

    # Create a black border around the zoomed-out image
    border_color = (0, 0, 0)  # Black color
    bordered_image = cv2.copyMakeBorder(image, border_height, border_height, border_width, border_width,
                                        cv2.BORDER_CONSTANT, value=border_color)
    zoomed_out_image = cv2.resize(bordered_image, (width, height))
    return zoomed_out_image

def augmentation_zoomout(image):
    array = [0.8,0.7]
    image_list = []
    for element in array:
        augmented_image = zoomout(image, element)
        image_list.append(augmented_image)
    return image_list


In [ ]:
def augmentation_meta(image):
    # Adjust brightness and contrast of the image
    array = np.arange(0.5, 2.5, 0.5)
    image_list = []
    for element in array:
        brightness = element  # Brightness factor (0.0 to 1.0)
        contrast = element + 1  # Contrast factor (>1.0 for higher contrast)
        augmented_image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
        image_list.append(augmented_image)
    return image_list

In [ ]:
def augmented_background(frame_aug, file_num, frame_number):
    frameaug = frame_aug
    seq_num = file_num
    augmented_rotation = augmentation_rotation(frameaug, 45)
    augmented_zoomin = augmentation_zoomin(frameaug)
    augmented_zoomout = augmentation_zoomout(frameaug)
    augmented_meta = augmentation_meta(frameaug)
    concated_image = augmented_rotation + augmented_zoomin + augmented_zoomout  + augmented_meta
    for  index, aug_image in enumerate(concated_image):
        image, results = mediapipe_detection(aug_image, holistic)
        keypoints = extract_keypoints(results)
        npy_path = os.path.join(DATA_PATH, action, str(seq_num), str(frame_number))
        print(npy_path)
        cv2.imwrite(f'{npy_path}.jpg', aug_image)
        np.save(npy_path, keypoints)
        seq_num+= 1

In [ ]:
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence = 0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        seq_num = 30
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through sequence length aka video length
            for frame_num in range(sequence_length):
                # Read feed
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                
                frame = cv2.imread(f'{npy_path}.jpg')
                augmented_background(frame,seq_num, frame_num)
            # Break gracefully
            seq_num += 12

# 7. Preprocess Data and Create Labels and Features

#### Dataset untuk testing digunakan sejumlah 5% dari dataset total

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 8. Build and Train BiGRU Neural Network

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(Bidirectional(GRU(64, return_sequences=True, activation='relu', input_shape=(30,126))))
model.add(Bidirectional(GRU(128, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=120, callbacks=[tb_callback], validation_split = 0.1, batch_size=16)

In [ ]:
model.summary()

In [ ]:
# Plot training & validation loss values
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(y_test[0])]

# 9. Save Weights

In [ ]:
model.save('SIBI_24Agustus2024_V2.h5')

In [ ]:
model.load_weights('SIBI_24Agustus2024_V2.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
# define class labels
class_labels = ['Tolong', 'Terima Kasih', 'Maaf']

# ytrue and yhat are the predicted and the actual labels
conf_matrix = confusion_matrix(ytrue, yhat)

accuracy = accuracy_score(ytrue, yhat)
precision = precision_score(ytrue, yhat, average='weighted')
recall = recall_score(ytrue, yhat, average='weighted')
f1 = f1_score(ytrue, yhat, average='weighted')

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Calculate classification report
report = classification_report(ytrue, yhat, target_names=class_labels)
print("\nClassification Report:\n", report)

# 11. Test in Real Time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        # MAC OS FRAME RECTANGLE
        
        # cv2.rectangle(image, start_point, end_point, color, thickness)
        # cv2.putText(image, text, org (coordinates), font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
        
        # Pembesaran Frame Rectangle = WORKED but masih KECIL
        cv2.rectangle(output_frame, (0, 200+num*200), (int(prob*480), 300+num*200), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 260+num*200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 5, cv2.LINE_AA)

        # Kasih Tambahan Angka Probabilitas 
        cv2.rectangle(output_frame, (500, 200+num*200), (650, 300+num*200), (0, 0, 0), 3)
        cv2.putText(output_frame, ' ' + str(round(prob*100, 1)) + '%', (500, 260+num*200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            # # Filter out the noise
            if np.argmax(np.bincount(predictions[-10:]))==np.argmax(res):
            # if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (2000, 150), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (30,100), 
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Hazlan SIBI REAL-TIME TEST | WEBCAM', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# 12. Bonus Step Perhitungan Arsitektur BiGRU

In [ ]:
import torch
import torch.nn.functional as F

# Input: Single keypoint value 0.1
X = torch.tensor([[0.5]])

# Define the GRU network parameters:
hidden_size = 1  # GRU layer with only one unit

### 2. First BiGRU Layer
# Shared weights for update, reset, and new state gates (for simplicity)
W_u = torch.tensor([[0.5]])
W_r = torch.tensor([[0.5]])
W = torch.tensor([[0.5]])  # W for candidate hidden state
Q_u = torch.tensor([[0.2]])
Q_r = torch.tensor([[0.2]])
Q = torch.tensor([[0.2]])  # Q for candidate hidden state
b_u = torch.tensor([0.1])
b_r = torch.tensor([0.1])
b = torch.tensor([0.1])    # b for candidate hidden state

# Initialize the hidden state for the forward and backward cells of the first BiGRU layer
h_t_f1 = torch.zeros(1)  # Forward cell
h_t_b1 = torch.zeros(1)  # Backward cell

# GRU cell calculation for the first BiGRU layer - forward cell
u_f1 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_f1, Q_u) + b_u)
r_f1 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_f1, Q_r) + b_r)
h_tilde_f1 = torch.tanh(torch.matmul(X, W) + b + r_f1 * (torch.matmul(h_t_f1, Q) + b))
h_t_f1_new = u_f1 * h_t_f1 + (1 - u_f1) * h_tilde_f1

# GRU cell calculation for the first BiGRU layer - backward cell
u_b1 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_b1, Q_u) + b_u)
r_b1 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_b1, Q_r) + b_r)
h_tilde_b1 = torch.tanh(torch.matmul(X, W) + b + r_b1 * (torch.matmul(h_t_b1, Q) + b))
h_t_b1_new = u_b1 * h_t_b1 + (1 - u_b1) * h_tilde_b1

# Combine the forward and backward hidden states using element-wise multiplication
h_t_1 = h_t_f1_new * h_t_b1_new

# Apply ReLU activation after combining
h_t_1_relu = F.relu(h_t_1)

### 3. Second BiGRU Layer
# Initialize the hidden state for the forward and backward cells of the second BiGRU layer
h_t_f2 = torch.zeros(1)  # Forward cell
h_t_b2 = torch.zeros(1)  # Backward cell

# GRU cell calculation for the second BiGRU layer - forward cell
u_f2 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_1_relu, Q_u) + b_u)
r_f2 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_1_relu, Q_r) + b_r)
h_tilde_f2 = torch.tanh(torch.matmul(X, W) + b + r_f2 * (torch.matmul(h_t_1_relu, Q) + b))
h_t_f2_new = u_f2 * h_t_1_relu + (1 - u_f2) * h_tilde_f2

# GRU cell calculation for the second BiGRU layer - backward cell
u_b2 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_1_relu, Q_u) + b_u)
r_b2 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_1_relu, Q_r) + b_r)
h_tilde_b2 = torch.tanh(torch.matmul(X, W) + b + r_b2 * (torch.matmul(h_t_1_relu, Q) + b))
h_t_b2_new = u_b2 * h_t_1_relu + (1 - u_b2) * h_tilde_b2

# Combine the forward and backward hidden states using element-wise multiplication
h_t_2 = h_t_f2_new * h_t_b2_new

# Apply ReLU activation after combining
h_t_2_relu = F.relu(h_t_2)

### 4. Dense Layer
# Dense layer weights and bias
W_dense = torch.tensor([[0.7]])
b_dense = torch.tensor([0.2])

# Dense layer output
y_linear = torch.matmul(h_t_2_relu, W_dense.T) + b_dense

# Apply ReLU activation after the dense layer
h_dense_relu = F.relu(y_linear)

### 5. Output Layer
# Softmax activation for output layer
y_linear_2 = torch.matmul(h_dense_relu, W_dense.T) + b_dense

y_pred = F.softmax(y_linear_2, dim=0)

# Print detailed calculations
print("\n=== First BiGRU Layer - Forward Cell ===")
print(f"Update Gate (u_f1): {u_f1.item()}")
print(f"Reset Gate (r_f1): {r_f1.item()}")
print(f"Candidate Hidden State (h_tilde_f1): {h_tilde_f1.item()}")
print(f"New Hidden State (h_t_f1_new): {h_t_f1_new.item()}")

print("\n=== First BiGRU Layer - Backward Cell ===")
print(f"Update Gate (u_b1): {u_b1.item()}")
print(f"Reset Gate (r_b1): {r_b1.item()}")
print(f"Candidate Hidden State (h_tilde_b1): {h_tilde_b1.item()}")
print(f"New Hidden State (h_t_b1_new): {h_t_b1_new.item()}")

print(f"\n1st First Cell ReLU: {h_t_1_relu.item()}")

print("\n=== Second BiGRU Layer - Forward Cell ===")
print(f"Update Gate (u_f2): {u_f2.item()}")
print(f"Reset Gate (r_f2): {r_f2.item()}")
print(f"Candidate Hidden State (h_tilde_f2): {h_tilde_f2.item()}")
print(f"New Hidden State (h_t_f2_new): {h_t_f2_new.item()}")

print("\n=== Second BiGRU Layer - Backward Cell ===")
print(f"Update Gate (u_b2): {u_b2.item()}")
print(f"Reset Gate (r_b2): {r_b2.item()}")
print(f"Candidate Hidden State (h_tilde_b2): {h_tilde_b2.item()}")
print(f"New Hidden State (h_t_b2_new): {h_t_b2_new.item()}")

print(f"\n 2nd Second Cell ReLU: {h_t_2_relu.item()}")

print("\n=== First Dense Layer ===")
print(f"Linear Output (y_linear): {y_linear.item()}")

print("\n=== Second Dense Layer (Output) ===")
print(f"2nd Linear Output (y_linear_2): {y_linear_2.item()}")
print(f"Softmax Output (y_pred): {y_pred.item()}")